In [6]:
import numpy as np
from utils import *

fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")

In [7]:
selected_feature_model = "fc_fd"

k = int(input("Enter value of k: "))
if k < 1:
    raise ValueError("k should be a positive integer")


label_mean_vectors = []



latent_semantics = extract_latent_semantics_from_feature_model(
    fd_collection,
    k,
    selected_feature_model,
)

U = np.array(latent_semantics["image-semantic"])
S = np.array(latent_semantics["semantics-core"])
if len(S.shape) == 1:
    S = np.diag(S)
V = np.transpose(np.array(latent_semantics["semantic-feature"]))
comparison_feature_space = np.matmul(U,S)

Applying svd on the fc_fd space to get 5 latent semantics.


In [8]:
true_positives = {}
false_negatives = {}
false_positives = {}
true_negatives = {}
for image_id in range(NUM_IMAGES):
    image = fd_collection.find_one({"image_id": (image_id*2+1)})
    comparison_vector = np.matmul(np.matmul(np.array(image[selected_feature_model]).flatten(), V), S)

    distances = []
    for i in range(NUM_LABELS):
        distances.append(
            {
                "image_id": image_id*2 + 1,
                "label": i,
                "distance": math.dist(
                    comparison_vector, comparison_feature_space[i]
                ),
            }
        )
    distances = sorted(distances, key=lambda x: x["distance"], reverse=False)
    # print(f"Similar Labels for image {(image_id*2+1)}")
    if distances[0]["label"] == image["true_label"]:
        true_positives[image["true_label"]] = true_positives.get(image["true_label"], 0) + 1
    else:
        false_negatives[image["true_label"]] = false_negatives.get(image["true_label"], 0) + 1
        false_positives[distances[0]["label"]] = false_positives.get(distances[0]["label"], 0) + 1


In [9]:
for i in range(NUM_LABELS):
    true_negatives[i] = NUM_IMAGES - (true_positives.get(i,0) + false_negatives.get(i,0) + false_positives.get(i,0))
    precision = true_positives.get(i,0)/(true_positives.get(i,0) + false_positives.get(i,0))
    recall = true_positives.get(i,0)/(true_positives.get(i,0)+false_negatives.get(i,0))
    if (precision + recall == 0):
        f1_score = None
    else:
        f1_score = 2*precision*recall/(precision+recall)    
    print(f"\nPrecision for Label {i}: {precision}")
    print(f"Recall for Label {i}: {recall}")
    print(f"F1 Score for Label {i}: {f1_score}")

accuracy = sum(true_positives.values())/NUM_IMAGES
print(f"Overall Accuracy: {accuracy}")

Precision for Label 0: 0.6772151898734177
Recall for Label 0: 0.4930875576036866
F1 Score for Label 0: 0.5706666666666667
Precision for Label 1: 0.62
Recall for Label 1: 0.7110091743119266
F1 Score for Label 1: 0.6623931623931624
Precision for Label 2: 0.5
Recall for Label 2: 0.23
F1 Score for Label 2: 0.31506849315068497
Precision for Label 3: 0.9184782608695652
Recall for Label 3: 0.42355889724310775
F1 Score for Label 3: 0.5797598627787307
Precision for Label 4: 0.19047619047619047
Recall for Label 4: 0.14814814814814814
F1 Score for Label 4: 0.16666666666666666
Precision for Label 5: 0.8899521531100478
Recall for Label 5: 0.465
F1 Score for Label 5: 0.6108374384236454
Precision for Label 6: 0.0
Recall for Label 6: 0.0
F1 Score for Label 6: None
Precision for Label 7: 0.06666666666666667
Recall for Label 7: 0.047619047619047616
F1 Score for Label 7: 0.05555555555555555
Precision for Label 8: 0.22727272727272727
Recall for Label 8: 0.20833333333333334
F1 Score for Label 8: 0.21739130